In [2]:
import torch
import torch.nn as nn

In [1]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")


In [8]:
A = torch.randint(0, 3, (2, 2, 2))
B = torch.randint(0, 3, (2, 2, 2))

In [13]:
A[1] @ B[1]

tensor([[0, 2],
        [2, 0]])

In [11]:
B

tensor([[[2, 1],
         [0, 1]],

        [[1, 0],
         [0, 1]]])

In [9]:
torch.bmm(A, B)

tensor([[[2, 3],
         [4, 4]],

        [[0, 2],
         [2, 0]]])

In [ ]:
lr_train_2 = 

In [6]:
lr_train.shape
lr_stack = lr_train.reshape(-1, 160)
print(lr_stack[0])

tensor([0.0000, 0.3388, 0.2025, 0.6895, 0.6105, 0.4464, 0.6495, 0.6894, 0.4389,
        0.5773, 0.0000, 0.4907, 0.2030, 0.5973, 0.1080, 0.5209, 0.2436, 0.3475,
        0.3142, 0.4680, 0.4328, 0.6836, 0.1751, 0.4156, 0.2886, 0.4879, 0.5107,
        0.0409, 0.6092, 0.3186, 0.3773, 0.5973, 0.4236, 0.5157, 0.0000, 0.2851,
        0.0880, 0.1500, 0.0387, 0.1536, 0.3912, 0.6067, 0.0019, 0.2677, 0.3644,
        0.1659, 0.2804, 0.0000, 0.2246, 0.0000, 0.0168, 0.0154, 0.2328, 0.1669,
        0.3741, 0.0000, 0.0000, 0.3887, 0.5526, 0.0068, 0.5261, 0.0912, 0.3366,
        0.1353, 0.0618, 0.1170, 0.3364, 0.1876, 0.2191, 0.0557, 0.0009, 0.1510,
        0.0694, 0.0450, 0.2801, 0.0509, 0.0000, 0.2265, 0.2669, 0.0807, 0.7052,
        0.1387, 0.1906, 0.3906, 0.0000, 0.2254, 0.4377, 0.2353, 0.0193, 0.2244,
        0.0000, 0.2057, 0.0000, 0.2625, 0.2219, 0.2037, 0.3883, 0.3881, 0.2607,
        0.1453, 0.0633, 0.1122, 0.2182, 0.0832, 0.2598, 0.3783, 0.2654, 0.1461,
        0.1611, 0.0000, 0.2819, 0.2389, 

In [15]:
lr_train2 = torch.bmm(lr_train, lr_train)
lr_stack2 = lr_train2.reshape(-1, 160)

In [16]:
end_dim = 16

class AutoencoderLR(nn.Module):
    def __init__(self):
        super(AutoencoderLR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(161, 64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32, end_dim))
        self.decoder = nn.Sequential(nn.Linear(end_dim, 32),
        nn.ReLU(),
        nn.Linear(32,64),
        nn.ReLU(),
        nn.Linear(64,160))
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops])
        x = self.encoder(x)
        return x
    def decode(latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x


In [17]:
end_dim = 16

class AutoencoderHR(nn.Module):
    def __init__(self):
        super(AutoencoderHR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(269, 64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32, end_dim))
        self.decoder = nn.Sequential(nn.Linear(end_dim, 32),
        nn.ReLU(),
        nn.Linear(32,64),
        nn.ReLU(),
        nn.Linear(64,160))
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops])
        x = self.encoder(x)
        return x
    def decode(latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x

In [ ]:
def train(criterion, model, adj_matrix, num_epoch = 50, lr = 0.01, batch_size = 16):
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    for step in num_epoch:
        for row in range(0,adj_matrix.shape[0], batch_size):
            optimizer.zero_grad()
            e = model(adj_matrix[row-batch_size:row], 1)
            loss = criterion(e, adj_matrix[row-batch_size:row])
            loss.backward()
            print(loss)
            optimizer.step()



In [ ]:
model = AutoencoderLR()
criterion = nn.MSELoss()
train(criterion, model, lr_stack)